In [2]:
from atproto import Client
import os

# 1. Connexion (L'auth n'est pas tjs requise pour la lecture publique,
# mais recommandée pour éviter les limites de débit)
client = Client()
# client.login('ton-handle.bsky.social', 'ton-mot-de-passe-app')

def track_post_info(post_uri):
    """
    Analyse les informations d'un post Bluesky et ses interactions
    """
    try:
        # Récupérer le thread (le post + les réponses)
        # depth=10 permet de descendre profondément dans l'arbre des réponses
        thread = client.get_post_thread(uri=post_uri, depth=10)
        
        post_data = thread.thread.post
        
        print(f"--- ANALYSE DU POST ---")
        print(f"Auteur: {post_data.author.handle}")
        print(f"Texte: {post_data.record.text}")
        print(f"Stats: {post_data.like_count} likes, {post_data.repost_count} reposts")
        
        # 2. Récupérer les gens qui ont reposté (les "vecteurs" de propagation)
        # Utiliser l'API Bluesky correctement
        try:
            reposts_response = client.app.bsky.feed.get_reposted_by({
                'uri': post_data.uri,
                'limit': 100  # Récupérer jusqu'à 100 profils
            })
            
            # Accéder à la liste des profils via 'reposted_by'
            reposted_by_list = reposts_response.reposted_by
            
            print(f"\n--- PROPAGATEURS (REPOSTS) - {len(reposted_by_list)} trouvés ---")
            if reposted_by_list:
                for profile in reposted_by_list:
                    display_name = profile.display_name if hasattr(profile, 'display_name') and profile.display_name else 'Sans nom'
                    followers = profile.followers_count if hasattr(profile, 'followers_count') else 'N/A'
                    print(f"- @{profile.handle} ({display_name}) - {followers} followers")
            else:
                print("Aucun repost trouvé")
        except Exception as repost_error:
            print(f"\n--- PROPAGATEURS (REPOSTS) ---")
            print(f"Erreur lors de la récupération: {repost_error}")
        
        # 3. Récupérer les réponses (le graphe de discussion)
        if hasattr(thread.thread, 'replies') and thread.thread.replies:
            print(f"\n--- RÉPONSES DIRECTES ---")
            for reply in thread.thread.replies:
                print(f"- De {reply.post.author.handle}: {reply.post.record.text[:50]}...")
        else:
            print("\nAucune réponse directe")
            
    except Exception as e:
        print(f"Erreur lors de la récupération du post: {e}")
        import traceback
        traceback.print_exc()

# Exemple d'utilisation avec un URI de post valide
# track_post_info('at://did:plc:z72i7hdynmk6r22z27h6tvur/app.bsky.feed.post/3lb6v...')

In [3]:
# Test avec le lien fourni
# L'authentification est requise pour pouvoir interroger les profils

# IMPORTANT: Pour fonctionner, vous devez vous authentifier
# Créez un "App Password" dans vos paramètres Bluesky si vous n'en avez pas

# Étape 1: Authentification
# Priorité : variable d'environnement > valeur par défaut codée en dur
HANDLE = os.getenv("BSKY_HANDLE", "for-sure.bsky.social")
APP_PASSWORD = os.getenv("BSKY_APP_PASSWORD", "LFfqA_n..7kz9KZ")

try:
    client.login(HANDLE, APP_PASSWORD)
    print(f"✓ Authentification réussie en tant que: {HANDLE}\n")
except Exception as e:
    print(f"✗ Erreur d'authentification: {e}")
    print("\nInstructions:")
    print("1. Remplissez HANDLE et APP_PASSWORD ci-dessus avec vos credentials")
    print("2. Créez un App Password dans: https://bsky.app/settings/app-passwords")
    print("3. Réexécutez cette cellule\n")

# Étape 2: Traitement du lien
def bsky_url_to_uri(url):
    """
    Convertit une URL Bluesky en URI AT valide
    Exemple: https://bsky.app/profile/mathieuhourdin.bsky.social/post/3mfetst7zbc2z
    """
    try:
        parts = url.split('/')
        handle = parts[4]  # mathieuhourdin.bsky.social
        post_id = parts[6]  # 3mfetst7zbc2z
        
        # Chercher le profil via search
        search_result = client.app.bsky.actor.search_actors({'term': handle})
        
        if search_result.actors:
            for actor in search_result.actors:
                if actor.handle == handle:
                    did = actor.did
                    uri = f"at://{did}/app.bsky.feed.post/{post_id}"
                    return uri
        
        return None
        
    except Exception as e:
        print(f"Erreur: {e}")
        return None

# Exécuter avec le lien fourni
bsky_url = "https://bsky.app/profile/mathieuhourdin.bsky.social/post/3mfetst7zbc2z"
print(f"Traitement de l'URL: {bsky_url}\n")
post_uri = bsky_url_to_uri(bsky_url)

if post_uri:
    print(f"✓ URI AT généré: {post_uri}\n")
    print("=== RÉSULTATS ===\n")
    track_post_info(post_uri)
else:
    print("✗ Impossible de générer l'URI AT")


✓ Authentification réussie en tant que: for-sure.bsky.social

Traitement de l'URL: https://bsky.app/profile/mathieuhourdin.bsky.social/post/3mfetst7zbc2z

✓ URI AT généré: at://did:plc:jyszdkkd7q4ejj4mah36jgde/app.bsky.feed.post/3mfetst7zbc2z

=== RÉSULTATS ===

--- ANALYSE DU POST ---
Auteur: mathieuhourdin.bsky.social
Texte: Bravo au journaliste de bfm sur place, qui répète à chaque fois qu'il prend la parole que les participants au rassemblement de Lyon essaient de masquer leur appartenance mais qu'il voit des croix gammées etc et que c'est des néonazis
Stats: 397 likes, 120 reposts

--- PROPAGATEURS (REPOSTS) - 100 trouvés ---
- @vincureuil.bsky.social (VincEcureuil) - N/A followers
- @muzahara3nour.bsky.social (Sans nom) - N/A followers
- @mikelkanik.bsky.social (Mikel K.) - N/A followers
- @legrand1775.bsky.social (Sans nom) - N/A followers
- @jeanlouv.bsky.social (jeanlouv) - N/A followers
- @noguesp.bsky.social (Nogues) - N/A followers
- @gorekhaa.bsky.social ([G.R.K.]©️😷👽🏴🏳️‍🌈

In [3]:
# Résumé concis des résultats
print("\n" + "="*50)
if post_uri:
    print("✓ RÉSUMÉ - Récupération terminée avec succès.")
else:
    print("✗ RÉSUMÉ - Échec de la récupération du post.")
print("="*50)



✓ RÉSUMÉ - Récupération terminée avec succès.


In [4]:
# Inspecter la structure de la réponse get_reposted_by
print("\n=== DEBUG: Structure de la réponse get_reposted_by ===")
try:
    debug_response = client.app.bsky.feed.get_reposted_by({
        'uri': 'at://did:plc:jyszdkkd7q4ejj4mah36jgde/app.bsky.feed.post/3mfetst7zbc2z',
        'limit': 5
    })
    print(f"Type: {type(debug_response)}")
    print(f"Attributs: {dir(debug_response)}")
    print(f"Contenu: {debug_response}")
except Exception as e:
    print(f"Erreur: {e}")


=== DEBUG: Structure de la réponse get_reposted_by ===
Type: <class 'atproto_client.models.app.bsky.feed.get_reposted_by.Response'>
Attributs: ['_ModelBase__alert_about_extra_fields', '__abstractmethods__', '__annotations__', '__class__', '__class_getitem__', '__class_vars__', '__copy__', '__deepcopy__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__fields__', '__fields_set__', '__format__', '__ge__', '__get_pydantic_core_schema__', '__get_pydantic_json_schema__', '__getattr__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__pretty__', '__private_attributes__', '__pydantic_complete__', '__pydantic_computed_fields__', '__pydantic_core_schema__', '__pydantic_custom_init__', '__pydantic_decorators__', '__pydantic_extra__', '__pydantic_fields__', '__pydantic_fields_set__', '__pydantic_generic_metadata__', '__pydantic_init_subclass__', '__pydantic

In [5]:
# Convertir en dictionnaire pour voir la structure
try:
    debug_response = client.app.bsky.feed.get_reposted_by({
        'uri': 'at://did:plc:jyszdkkd7q4ejj4mah36jgde/app.bsky.feed.post/3mfetst7zbc2z',
        'limit': 3
    })
    
    # Convertir en dict
    if hasattr(debug_response, 'model_dump'):
        data = debug_response.model_dump()
        print(f"Clés: {list(data.keys())}")
    else:
        print(f"Type: {type(debug_response)}")
        print(f"First 200 chars: {str(debug_response)[:200]}")
            
except Exception as e:
    print(f"Erreur: {e}")

Clés: ['reposted_by', 'uri', 'cid', 'cursor']
